In [ ]:
#This notebook is updated code that downloads videos from youtube, cuts them into clips where a touch occurs, and then correctly labels them as either left or right
#Special thanks to sholtodouglas and his repo (https://github.com/sholtodouglas/fencing-AI) for writing the original code 2 years ago 

In [ ]:
!pip install git+https://github.com/nficano/pytube
import pytube
from pytube import YouTube
import os
## Run this to download all the videos from youtube.
import signal
import time
import traceback
## Timeout for use with try/except so that pytube doesn't randomly freeze.
class Timeout():
    """Timeout class using ALARM signal."""
    class Timeout(Exception):
        pass
 
    def __init__(self, sec):
        self.sec = sec
 
    def __enter__(self):
        signal.signal(signal.SIGALRM, self.raise_timeout)
        signal.alarm(self.sec)
 
    def __exit__(self, *args):
        signal.alarm(0)    # disable alarm
 
    def raise_timeout(self, *args):
        raise Timeout.Timeout()

text = 'Phase 33 Data.txt'
phase_number = text.split(' ')[1]




# Create all the directories needed if they don't yet exist
# The only non automatic step is right before optical flow production. Some data will be in 'training quarantine', some in 'more_training data'
# (i.e, the extra augmented data). Up to the user to copy these over to final training_clips when ready. 
directories = ['precut','videos','training_quarantine','more_training_data',  'training_data', "Phase "+phase_number+" (30)/Right","Phase "+phase_number+" (30)/Left"]
for dirs in directories:
	if not os.path.exists(dirs):
	    os.makedirs(dirs)



print(phase_number)
text_file = open(text, "r")
vids = text_file.read().split('\n')
print("LOOK HERE")
print(len(vids))
print ("First 3 links:", vids[:3])
text_file.close()

# Loop through all the videos, download them and put them in the precut folder.
counter = 0
vids = vids[counter:]




for i in vids:
	
	
	try:
		with Timeout(600):
			video_url = i # paste here your Youube videos' url
			
			youtube = pytube.YouTube(video_url)
			video = youtube.streams.first()
			
			video.download(output_path="/content/precut", filename=str(counter)) # path, where to video download.
			
			print ("Downloaded: ", i,)
	except:
		
		traceback.print_exc()
		print ("Failed-",i)



	
	counter = counter + 1




  Cloning https://github.com/nficano/pytube to /tmp/pip-req-build-o9tyk9td
  Running command git clone -q https://github.com/nficano/pytube /tmp/pip-req-build-o9tyk9td
  Created wheel for pytube: filename=pytube-10.4.1-cp36-none-any.whl size=42912 sha256=1537b306f4c94fea511f2c1137fc42758f013ae13cd0dd44a3faeacb901664eb
  Stored in directory: /tmp/pip-ephem-wheel-cache-dp21oml2/wheels/44/da/40/3b5e03abe33a91895343814fb44b309512375408f4a909555b
Successfully built pytube
33
LOOK HERE
64
First 3 links: ['\ufeffhttps://www.youtube.com/watch?v=ZAaVH6aTGGo', 'https://www.youtube.com/watch?v=fdu_G8oI2Bc', 'https://www.youtube.com/watch?v=x2L0-pZvM6Q']
Downloaded:  ﻿https://www.youtube.com/watch?v=ZAaVH6aTGGo
Downloaded:  https://www.youtube.com/watch?v=fdu_G8oI2Bc
Downloaded:  https://www.youtube.com/watch?v=x2L0-pZvM6Q
Downloaded:  https://www.youtube.com/watch?v=EazzrB6-DkI
Downloaded:  https://www.youtube.com/watch?v=Tpq0IXJmmd0
Downloaded:  https://www.youtube.com/watch?v=OSVhj8Bfnus
Downlo

In [ ]:
!pip install opencv-python
import os
!rm -rf /content/videos
if not os.path.exists('videos'):
	os.makedirs('videos')

## Cuts the videos into a set of short clips where each actual hit happens. These clips are used by the data_labeller to 
## label the clips where the referee had to distinguish whos priority.
import cv2
import tensorflow as tf
import numpy as np
import argparse
import time
#import cv
import subprocess as sp
import os
from pylab import *
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
green_box = cv2.imread("greenbox.png")
red_box = cv2.imread("redbox.png")
white_box = cv2.imread("whitebox.png")



# Prelim info, FOTR light box is frame[329:334, 380:500]
# therefore FOTL box is frame[329:334, 140:260]
# FOTL OFF-TARGET frame[337:348, 234:250]
# FOTR OFF-TARGET frame[337:348, 390:406]
#      scoreleft = frame[310:325, 265:285]
#      scoreRight = frame[310:325, 355:375]
    
FFMPEG_BIN = "ffmpeg"


#!pip install -v scikit-learn==0.17.1


## Load a pretrained logisitc classifier which distinguishes between numbers 0-15.
import pickle

with open('logistic_classifier_0-15.pkl', 'rb') as fid:
  model = pickle.load(fid, encoding='latin1')


fps = str(30) ##### was 13
jump_length = 260 # this is how long our 'recording time' will be, where we don't check for lights, actual recording time, its so long because 
#we want to skip people testing their blades after hits #was 260
## is jump length - hide length = 'clip length'
hide_length = 201 # where we're not actually interested in keeping the frames, but don't want them to be seen by 'not in record mode' #was 200
video_number = 0

videos_to_cut = 0

for i in os.listdir(os.getcwd() + "/precut"):
    if i.endswith(".mp4"):
        videos_to_cut = videos_to_cut + 1


print ("Cutting", videos_to_cut, "videos")

already_processed = 0
!rm -rf /content/videos
if not os.path.exists('videos'):
	os.makedirs('videos')
for vid in os.listdir(os.getcwd() + "/precut"):
    
    if vid.endswith(".mp4") and int(vid.replace(".mp4","")) >= already_processed :
        print ("Video:", vid)
        clips_recorded = 0
        recording_mode = False
    
        cap = cv2.VideoCapture("precut/" + str(vid))
        
        cap_end_point = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print ("Length of Vid:", cap_end_point, int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
        cap.release()
        cap_end_point = cap_end_point - jump_length # ensures videos don't overrun
        print ("Beginning to cut...")
        
        position = 0

        while position < cap_end_point:
            cap = cv2.VideoCapture("precut/" + str(vid))

            
            if position == cap_end_point:
                print ("should not be here")
                break            

            
            if recording_mode == True:
                output_file = 'videos/' + str(vid).replace(".mp4","") +"-" + str(clips_recorded) + " (30)" + '.mp4'
                
           
                command = [FFMPEG_BIN,
                '-y',
                '-f', 'rawvideo',
                '-vcodec','rawvideo',
                '-s', '640*360',
                '-pix_fmt', 'bgr24',
                '-r', fps,
                '-i', '-',
                #'-an',
                #'-acodec', 'mp3',
                '-vcodec', 'mpeg4',
                '-b:v', '5000k',
                output_file ]

                frames_till_video_end = jump_length
                
                proc = sp.Popen(command, stdin=sp.PIPE, stderr=sp.PIPE)

            
            if cap.isOpened():

                cap.set(1,position)
                cap.set(cv2.CAP_PROP_FPS, 10000) 

                while cap.isOpened():
                    ret,frame = cap.read()
                    position = position + 1

                    

                    if recording_mode == False:
                        
                        if position % 100 == 0:
                            
                            print (cap.get(cv2.CAP_PROP_POS_FRAMES))
                            
                        # this check is here because the vid should be prevented from 
                        # starting a clip less than frames_till_vid_end away from 
                        # the end
                        if position == (cap_end_point):
                            break
                        elif cap.get(cv2.CAP_PROP_POS_FRAMES) >= cap_end_point:
                           
                            position = cap.get(cv2.CAP_PROP_POS_FRAMES)
                            break
                        try:
                            
                            if (np.sum(abs(frame[337:348, 234:250].astype(int)-white_box.astype(int))) <= 7000) or (np.sum(abs(frame[337:348, 390:406].astype(int)-white_box.astype(int))) <= 7000) or (np.sum(abs(frame[330:334, 380:500].astype(int)-green_box.astype(int))) <= 40000) or (np.sum(abs(frame[330:334, 140:260].astype(int)-red_box.astype(int))) <= 40000):
                                
                                left_score = model.predict(frame[309:325, 265:285].reshape(1,-1))
                                right_score = model.predict(frame[309:325, 355:375].reshape(1,-1))
                                
                        
                                if (left_score == 15) or (right_score == 15):
                                  
                                    position = position + 25
                                    break
                                elif (left_score == 0) and (right_score == 0):
                                    
                                    position = position + 25
                                    break
                                else:
                                    
                        
                                    ## jump back 50 frames to the action of the hit
                                    position = position - 50
                                    
                                    #print ("Light seen, position-", position)
                                    recording_mode = True
                                    break
                        except:
                            break
                    
                    
                    if recording_mode == True:
                        if frames_till_video_end >= hide_length:
                            #if position % 2 == 0:
                                proc.stdin.write(frame.tostring())
                        
                        frames_till_video_end = frames_till_video_end - 1
                        if frames_till_video_end == 0:
                            print ("finished clip")
                            recording_mode = False
                            proc.stdin.close()
                            proc.stderr.close()
                            print(clips_recorded)
                            clips_recorded = clips_recorded+1
                            break   
            else:
                print("Failed to open video")

            cap.release()
            video_number = video_number + 1



Cutting 64 videos


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version pre-0.18 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Video: 16.mp4
Length of Vid: 38936 38936
Beginning to cut...
100.0
200.0
300.0
400.0
500.0
600.0
700.0
800.0
900.0
1000.0
1100.0
1200.0
1300.0
1400.0
1500.0
1600.0
1700.0
1800.0
1900.0
2000.0
2100.0
2200.0
2300.0
2400.0
2500.0
2600.0
2700.0
2800.0
2900.0
3000.0
3100.0
3200.0
3300.0
3400.0
3500.0
3600.0
3700.0
3800.0
3900.0
4000.0
4100.0
4200.0
4300.0
4400.0
4500.0
4600.0
4700.0
4800.0
4900.0
5000.0
5100.0
5200.0
5300.0
5400.0
5500.0
5600.0
5700.0
5800.0
5900.0
6000.0
6100.0
6200.0
6300.0
6400.0
6500.0
6600.0
6700.0
6800.0
6900.0
7000.0
7100.0
7200.0
7300.0
7400.0
7500.0
7600.0
7700.0
7800.0
7900.0
8000.0
8100.0
8200.0
8300.0
8400.0
8500.0
8600.0
8700.0
8800.0
8900.0
9000.0
9100.0
9200.0
9300.0
9400.0
9500.0
9600.0
9700.0
9800.0
9900.0
10000.0
10100.0
10200.0
10300.0
10400.0
10500.0
10600.0
10700.0
10800.0
10900.0
11000.0
11100.0
11200.0
11300.0
11400.0
11500.0
11600.0
11700.0
11800.0
11900.0
12000.0
12100.0
12200.0
12300.0
12400.0
12500.0
12600.0
12700.0
12800.0
12900.0
13000.0
13100.0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:178: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


finished clip
0
7300.0
7400.0
7500.0
7600.0
7700.0
7800.0
7900.0
8000.0
8100.0
8200.0
8300.0
8400.0
8500.0
8600.0
8700.0
8800.0
8900.0
9000.0
9100.0
9200.0
9300.0
9400.0
9500.0
9600.0
9700.0
9800.0
9900.0
10000.0
10100.0
10200.0
10300.0
10400.0
10500.0
10600.0
10700.0
10800.0
10900.0
11000.0
11100.0
11200.0
11300.0
11400.0
11500.0
11600.0
11700.0
11800.0
11900.0
12000.0
12100.0
12200.0
12300.0
12400.0
12500.0
12600.0
12700.0
12800.0
12900.0
13000.0
13100.0
13200.0
13300.0
13400.0
13500.0
13600.0
13700.0
13800.0
13900.0
14000.0
14100.0
14200.0
14300.0
14400.0
14500.0
14600.0
14700.0
14800.0
14900.0
15000.0
15100.0
15200.0
15300.0
15400.0
15500.0
15600.0
15700.0
15800.0
15900.0
16000.0
16100.0
16200.0
16300.0
16400.0
16500.0
16600.0
16700.0
16800.0
16900.0
17000.0
17100.0
17200.0
17300.0
17400.0
17500.0
17600.0
17700.0
17800.0
17900.0
18000.0
18100.0
18200.0
18300.0
18400.0
18500.0
18600.0
18700.0
18800.0
18900.0
19000.0
19100.0
19200.0
19300.0
19400.0
19500.0
19600.0
19700.0
19800.0
199

...

finished clip
2
12000.0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:178: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


finished clip
3
12300.0
12400.0
12500.0
12600.0
12700.0
12800.0
12900.0
13000.0
13100.0
13200.0
13300.0
13400.0
13500.0
13600.0
13700.0
13800.0
13900.0
14000.0
14100.0
14200.0
14300.0
14400.0
14500.0
14600.0
14700.0
14800.0
14900.0
15000.0
15100.0
15200.0
15300.0
15400.0
15500.0
15600.0
15700.0
15800.0
15900.0
16000.0
16100.0
16200.0
16300.0
16400.0
16500.0
16600.0
16700.0
16800.0
16900.0
17000.0
17100.0
17200.0
17300.0
17400.0
17500.0
17600.0
17700.0
17800.0
17900.0
18000.0
18100.0
18200.0
18300.0
18400.0
18500.0
18600.0
18700.0
18800.0
18900.0
19000.0
19100.0
19200.0
19300.0
19400.0
19500.0
19600.0
19700.0
19800.0
19900.0
20000.0
20100.0
20200.0
20300.0
20400.0
20500.0
20600.0
20700.0
20800.0
20900.0
21000.0
21100.0
21200.0
21300.0
21400.0
21500.0
21600.0
21700.0
21800.0
21900.0
22000.0
22100.0
22200.0
22300.0
22400.0
22500.0
22600.0
22700.0
22800.0
22900.0
23000.0
23100.0
23200.0
23300.0
23400.0
23500.0
23600.0
23700.0
23800.0
23900.0
24000.0
24100.0
24200.0
24300.0
24400.0
24500.0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:178: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


finished clip
0
22600.0
22700.0
22800.0
22900.0
23000.0
23100.0
23200.0
23300.0
23400.0
23500.0
23600.0
23700.0
23800.0
23900.0
Video: 52.mp4
Length of Vid: 36188 36188
Beginning to cut...
100.0
200.0
300.0
400.0
500.0
600.0
700.0
800.0
900.0
1000.0
1100.0
1200.0
1300.0
1400.0
1500.0
1600.0
1700.0
1800.0
1900.0
2000.0
2100.0
2200.0
2300.0
2400.0
2500.0
2600.0
2700.0
2800.0
2900.0
3000.0
3100.0
3200.0
3300.0
3400.0
3500.0
3600.0
3700.0
3800.0
3900.0
4000.0
4100.0
4200.0
4300.0
4400.0
4500.0
4600.0
4700.0
4800.0
4900.0
5000.0
5100.0
5200.0
5300.0
5400.0
5500.0
5600.0
5700.0
5800.0
5900.0
6000.0
6100.0
6200.0
6300.0
6400.0
6500.0
6600.0
6700.0
6800.0
6900.0
7000.0
7100.0
7200.0
7300.0
7400.0
7500.0
7600.0
7700.0
7800.0
7900.0
8000.0
8100.0
8200.0
8300.0
8400.0
8500.0
8600.0
8700.0
8800.0
8900.0
9000.0
9100.0
9200.0
9300.0
9400.0
9500.0
9600.0
9700.0
9800.0
9900.0
10000.0
10100.0
10200.0
10300.0
10400.0
10500.0
10600.0
10700.0
10800.0
10900.0
11000.0
11100.0
11200.0
11300.0
11400.0
11500.0

In [ ]:
## Looks at each clip, if there were two lights, look at the next clip to see what changes there were in the scoreboard.
## Depending on the changes in the scoreboard, label the clip according. If there weren't two lights, disregard the clip 
## as it doesn't provide as any more information. 

import cv2
import tensorflow as tf
import numpy as np
import argparse
import time

#import cv
import subprocess as sp
import os
from pylab import *
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
green_box = cv2.imread("greenbox.png")
red_box = cv2.imread("redbox.png")
white_box = cv2.imread("whitebox.png")


import pickle

with open('logistic_classifier_0-15.pkl', 'rb') as fid:
  model = pickle.load(fid, encoding='latin1')

################################################################################################

def check_lights(frame):
    # returns a string, either On-On, On-Off, Off-On, Off-Off, On-No, No-On, Off-No, No-Off
    
    # red is on the left, green on the right
    leftOff = False
    leftOn = False
    rightOff = False
    rightOn = False
    string = ""
     #check for left on target light
    if (np.sum(abs(frame[330:334, 140:260].astype(int)-red_box.astype(int))) <= 40000):
        string = string + "On"
    #check for left off target light
    elif (np.sum(abs(frame[337:348, 234:250].astype(int)-white_box.astype(int))) <= 7000):
        string = string + "Off"
    else:
        string = string + "No"
        
    #check for right off target light
    string = string + "-"
    if (np.sum(abs(frame[330:334, 380:500].astype(int)-green_box.astype(int))) <= 40000):
    
       string = string + "On"
    #ccheck for right on target light
    elif (np.sum(abs(frame[337:348, 390:406].astype(int)-white_box.astype(int))) <= 7000):
        string = string + "Off"
    else:
        string = string + "No"
        
    return string
 ################################################################################################

def check_score(frame):
     if frame is not None:
      left = model.predict(frame[309:325, 265:285].reshape(1,-1)) 
      right = model.predict(frame[309:325, 355:375].reshape(1,-1))
      return left, right
################################################################################################

def caption(hit_type,left,right,update_left,update_right):
    caption = "None"
    if hit_type == "On-On":
        if update_left-left == 1 and update_right-right == 0:
            caption = "L"
        if update_left-left == 0 and update_right-right == 1:
            caption = "R"
        if update_left-left == 0 and update_right-right == 0:
            caption = "T"
    if hit_type == "On-Off":
        if update_left-left == 1 and update_right-right == 0:
            caption = "L"
        if update_left-left == 0 and update_right-right == 0:
            caption = "R"
    if hit_type == "Off-On":
        if update_left-left == 0 and update_right-right == 1:
            caption = "R"
        if update_left-left == 0 and update_right-right == 0:
            caption = "L"
            
    return caption

################################################################################################

for i in os.listdir(os.getcwd() + "/videos"):
    if i.endswith(".mp4"): 
        

        match_number = int(i.split("-")[0])

        hit_number = int((i.split("-")[1].split(" ")[0].replace(".mp4","")))


        print ("Match-Hit",match_number, hit_number)
        cap = cv2.VideoCapture("videos/" + i)
        cap_end_point = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        cap.set(1,cap_end_point-1)  
        ret,frame = cap.read()
        if frame is not None:
          hit_type = check_lights(frame)
          left,right = check_score(frame)

        
        cap.release()

        if hit_type == "On-On" or hit_type == "On-Off" or hit_type == "Off-On":
        #### now open the following hit
            next_hit = "videos/" + str(match_number) + "-" + str(hit_number+1) + " (30)"+ ".mp4"
            if os.path.isfile(next_hit) == True:
                # open the next hit
                cap = cv2.VideoCapture(next_hit)
                cap.set(1,0)
                ret,frame = cap.read()
                if frame is not None:
                  update_left,update_right = check_score(frame)
                  cap.release()
                  priority = caption(hit_type,left,right,update_left,update_right)

                
                print (priority)
                if priority != 'None':
                    os.rename("videos/"+ i, "training_quarantine/"+priority+i)


        continue
    else:
        continue

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version pre-0.18 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Match-Hit 20 0
Match-Hit 26 7
Match-Hit 19 5
Match-Hit 26 5
Match-Hit 4 0
Match-Hit 45 1
Match-Hit 19 0
Match-Hit 19 6
Match-Hit 19 2
Match-Hit 26 6
Match-Hit 26 8
Match-Hit 19 1
Match-Hit 19 4
Match-Hit 23 1
Match-Hit 26 3
Match-Hit 26 1
Match-Hit 22 0
Match-Hit 56 0
Match-Hit 45 0
Match-Hit 24 1
Match-Hit 20 1
Match-Hit 24 3
Match-Hit 26 2
Match-Hit 24 2
Match-Hit 22 1
Match-Hit 26 0
Match-Hit 20 2
Match-Hit 20 3
Match-Hit 3 0
Match-Hit 23 2
Match-Hit 46 0
Match-Hit 45 2
Match-Hit 61 0
Match-Hit 23 0
Match-Hit 26 4
Match-Hit 19 3
Match-Hit 24 0


In [ ]:
moved=0
for i in os.listdir(os.getcwd() + "/training_quarantine"):
  if i[0] == 'L':
    os.rename("/content/training_quarantine/"+i,"/content/Phase "+phase_number+" (30)/Left/"+phase_number+'-'+i)
    moved=moved+1
  elif i[0] == 'R':
    os.rename("/content/training_quarantine/"+i,"/content/Phase "+phase_number+" (30)/Right/"+phase_number+'-'+i)
    moved=moved+1
  
print("Total files moved: "+str(moved))


Total files moved: 0


In [ ]:
!zip -r "Phase 33 (30).zip" "/content/Phase 33 (30)"
from google.colab import files
files.download("Phase "+phase_number+' (30).zip')

  adding: content/Phase 33 (30)/ (stored 0%)
  adding: content/Phase 33 (30)/Left/ (stored 0%)
  adding: content/Phase 33 (30)/Right/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>